In [20]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

C:\Users\kapronczaym\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\kapronczaym\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\kapronczaym\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [84]:
class CausalNode:

    def __init__(self, x, y, idxs, treatment, min_leaf=5):
        self.x = x 
        self.y = y
        self.idxs = idxs 
        self.min_leaf = min_leaf
        self.row_count = len(idxs)
        self.col_count = x.shape[1]
        self.val = np.mean(y[idxs])
        self.score = float('-inf')
        self.treatment = treatment
        self.find_varsplit()
        
    def find_varsplit(self):
        for c in range(self.col_count): self.find_better_split(c)
        if self.is_leaf: return
        x = self.split_col
        lhs = np.nonzero(x <= self.split)[0]
        rhs = np.nonzero(x > self.split)[0]
        self.lhs = CausalNode(self.x, self.y, self.idxs[lhs], self.treatment[lhs], self.min_leaf)
        self.rhs = CausalNode(self.x, self.y, self.idxs[rhs], self.treatment[rhs], self.min_leaf)
        
    def find_better_split(self, var_idx):
      
        x = self.x.values[self.idxs, var_idx]

        for r in range(self.row_count):
            lhs = x <= x[r]
            rhs = x > x[r]
            if rhs.sum() < self.min_leaf or lhs.sum() < self.min_leaf: continue

            curr_score = self.find_score(lhs, rhs)
            if curr_score > self.score: 
                self.var_idx = var_idx
                self.score = curr_score
                self.split = x[r]
                
    def find_score(self, lhs, rhs):
        y = self.y[self.idxs]
        y_treat = np.mean(y[self.treatment == 1])
        y_control = np.mean(y[self.treatment == 0])
        return (y_treat - y_control) ** 2
                
    @property
    def split_col(self): return self.x.values[self.idxs,self.var_idx]
                
    @property
    def is_leaf(self): return self.score == float('-inf')                

    def predict(self, x):
        return np.array([self.predict_row(xi) for xi in x])

    def predict_row(self, xi):
        if self.is_leaf: return self.val
        node = self.lhs if xi[self.var_idx] <= self.split else self.rhs
        return node.predict_row(xi)
    
class CausalTreeRegressor:
  
  def fit(self, X, y, treatment, min_leaf = 5):
    self.dtree = CausalNode(X, y, np.array(np.arange(len(y))), treatment, min_leaf)
    return self

  def predict(self, X):
    return self.dtree.predict(X.values)

In [85]:
ct = CausalTreeRegressor().fit(X=pd.DataFrame(data), y=target, treatment=data[:, 3], min_leaf=10)

C:\Users\kapronczaym\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\kapronczaym\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [86]:
ct.predict(pd.DataFrame(pd.DataFrame(data[0, :]).T))

array([32.67142857])

In [87]:
target[0]

24.0

In [88]:
ct.predict(pd.DataFrame(pd.DataFrame(data[1, :]).T))

array([29.18181818])

In [89]:
target[1]

21.6

In [79]:
pd.DataFrame(data[0, :]).T

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.09,1.0,296.0,15.3,396.9,4.98


In [60]:
dt = DecisionTreeRegressor().fit(pd.DataFrame(data), target)

In [22]:
from sklearn.datasets import load_boston

In [40]:
print(boston.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

In [38]:
boston = load_boston()

In [24]:
data = boston['data']

In [25]:
target = boston['target']

In [29]:
dt = DecisionTreeRegressor().fit(X=pd.DataFrame(data), y=target)

In [31]:
dt.predict(pd.DataFrame(data))

array([27.35555556, 21.54444444, 35.36      , 35.36      , 33.82      ,
       24.7       , 19.2       , 21.5       , 17.81666667, 21.5       ,
       17.81666667, 19.2       , 22.24      , 18.5       , 19.2       ,
       20.44      , 22.1       , 17.88333333, 22.56666667, 17.88333333,
       13.75      , 19.94285714, 15.88333333, 13.75      , 15.88333333,
       16.03333333, 18.        , 14.56      , 19.2       , 19.94285714,
       13.75      , 17.        , 12.48571429, 14.56      , 12.48571429,
       18.5       , 24.86      , 20.44      , 22.95      , 28.14285714,
       36.1       , 28.14285714, 23.68      , 23.68      , 23.68      ,
       19.2       , 18.        , 19.06666667, 17.81666667, 20.05      ,
       19.2       , 21.54444444, 26.36      , 22.1       , 18.        ,
       33.78      , 24.7       , 30.4       , 23.12      , 19.55      ,
       18.        , 19.2       , 23.48      , 25.975     , 31.5       ,
       23.28888889, 19.2       , 22.1       , 18.        , 22.1 

In [3]:
DecisionTreeRegressor

__main__.DecisionTreeRegressor